# Еще несколько функций высших порядков
## Функция композиции (.)
$$h(x) = f(g(x))$$
$$h = f \circ g$$

In [1]:
-- Настоящая композиция называется . , а мы заведем .*
(.*) :: (b -> c) -> (a -> b) -> (a -> c)
f .* g = \x -> f (g x)

In [2]:
let h = (+1) .* (*2) in h 10
let h = (*2) .* (+1) in h 10

21

22

In [3]:
:t (.) -- показать тип выражения. Действительно, композиция имеет такой же тип, как придумали мы

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

## Функция применения (\$)

`f $ x = f(x)`

In [4]:
($*) :: (a -> b) -> a -> b
f $* x = f x

Пример использования, избегание скобок для группировки аргумента функции

In [5]:
fact :: Int -> Int
fact 0 = 1
fact n = n * (fact $ n - 1)
-- fact n = n * fact (n - 1) -- еще можно так

fact 5

120

# Алгебраические типы данных

In [6]:
-- типы и конструкторы типов обязаны называться с большой буквы
data Mark = Unsatisfactory | Satisfactory | Good | Excellent
data Sex = Male | Female
data Human = Student [Char] Int | Lecturer [Char] -- [Char] это имя, а Int - номер курса

-- используем созданные только что типы
mathMark = Good
:t mathMark

mathMark :: Mark

Мы завели *типы* `Mark`, `Sex`, `Human`, а `Unsatisfactory`, `Good`, `Male`, `Student` и т.д. - это *конструкторы типов*.

In [7]:
-- лучше писать c deriving Show, чтобы можно было распечатывать значения типа для тестирования
data Human = Student String Int | Lecturer String deriving Show

In [8]:
s1 = Student "Ilya" 4
l1 = Lecturer "Posov"
:t s1
:t l1

s1 :: Human

l1 :: Human

Значения `l1` и `s1` имеют тип `Human`. Типов `Student` и `Lecturer` не существует, это конструкторы, которые позволяют создавать значения типа `Human`

In [9]:
s1

Student "Ilya" 4

Как же использовать значения этих типов и узнавать, что у них внутри? Основной способ использования - сопоставлять с образцом.

In [10]:
-- функция красивого вывода на печать
showHuman :: Human -> [Char]
showHuman (Student name course) = "Student " ++ name ++ " from course " ++ (show course)
showHuman (Lecturer name) = "Lecturer " ++ name

showHuman s1
showHuman l1

isStudent :: Human -> Bool
isStudent (Student _ _) = True
isStudent _ = False

-- вспомним, что _ означает, что нас не интересует конкретное значение

isStudent s1
isStudent l1

"Student Ilya from course 4"

"Lecturer Posov"

True

False

## Рекурсивные алегбраические типы
Заведем тип для хранения списка чисел (Int). Т.е. повторим встроенный тип данных `[]`

In [11]:
data List = Empty | Cons Int List -- рекурсивность в том, что мы использовали List при описании List
-- лишний раз обратим внимание
-- в записи Cons Int List первое - это конструктор типа, а остальное (Int, List) это типы тех значений,
-- которые будут использоваться в конструкторе:

l1 = Cons 4 (Cons 2 Empty) -- создал "список" [4, 2]
l2 = 1 `Cons` (2 `Cons` (3 `Cons` Empty)) -- [1, 2, 3, 4]

Можно повторить все функции по работе со списками для наших новых списков. Только теперь нужно будет использовать `Cons` вместо `:`, `Empty` вместо `[]`.

Заведем теперь тип "двоичное дерево". Дерево состоит из узлов, в каждом узле записано числовое значение. У узлов могут быть дочерние узлы. Если дочерних узлов нет, такой узел называется листом. В двоичном дереве не в листьях всегда ровно 2 дочерних узла.

Попробуем сохранить следующее дерево:

```
      3
      |
   -------
   |     |
   7     5
   |
-------
|     |
5     6
```

In [12]:
data Tree = Leaf Int | Node Int Tree Tree
-- дерево это либо лист со значением, либо узел со значением и двуми дочерними узлами.
tree = Node 3 (Node 7 (Leaf 5) (Leaf 6)) (Leaf 5)

In [13]:
-- сумма элементов дерева
sumTree :: Tree -> Int
sumTree (Leaf v) = v
sumTree (Node v left right) = v + (sumTree left) + (sumTree right)

Замечание. Конструкторы типов можно понимать и даже использовать как функции. Что такое, например, `Leaf`, это функция типа Int -> Tree. А `Student :: [Char] -> Int -> Human`

In [14]:
:t Leaf
:t Student

Leaf :: Int -> Tree

Student :: String -> Int -> Human

Еще замечание. Если у типа есть только один конструктор, его принято называть так же как и сам тип. Хотя это разные вещи, они используются в разных контекстах.

In [15]:
data Rectangle = Rectangle Int Int -- прямоугольник с высотой и шириной
-- первый Rectangle - это тип
-- второй Rectangle - это конструктор типа

area :: Rectangle -> Int
area (Rectangle w h) = w * h

-- аналогично, в описании типа area, Rectangle обозначает тип.
-- В сопоставлении с образцом это конструктор типа.

## Параметризованные алгебраические типы
Как сделать так, чтобы описанный нами List мог быть не только из Int, но из любого типа

In [16]:
data List a = Empty | Cons a (List a)

Т.е. тип List принимает типовый аргумент a, это тот тип, из чего состоит список. (Обычно мы использовали `[a]`, а в нашем списке надо писать `List a`)

# Обработка ошибок в Haskell
Сейчас мы рассмотрим простые методы работы с ошибками, а аналоги исключений будут позже, когда мы изучим монады.

## Тип Maybe
Он определяется в стандартной библиотеке Haskell (в модуле Prelude)

In [17]:
data Maybe' a = Just' a | Nothing'

Значение типа, например, `Maybe Int` это либо ничего, либо число. Представьте, что есть список из Int, т.е. `[Int]` и вы хотите получить его голову. В обычной ситуации вы вызовите функцию `head`, она вернет `Int`, но если головы нет, как у пустого списка, возникнет ошибка. Поэтому лучше сделать вот такую версию `head`:

In [18]:
headMayBe :: [a] -> Maybe a
headMayBe [] = Nothing
headMayBe (x:_) = Just x

Эта версия функции `head` никогда не сломается на пустом списке

In [19]:
headMayBe [] -- здесь будет Nothing
headMayBe [1, 2, 3] -- здесь будет Just 1

Nothing

Just 1

В Java в последних версиях появился тип Optional<>, у него есть два подкласса Some и None. Он имеет ровно такой же смысл, вы можете его использовать вместо null. Обычно null означает отсутствие значения. Но с null много проблем, потому что нужно постоянно не забывать проверять значение на null. Теперь, если вы хотите сказать, что функция не возвращает значения, верните лучше Nothing (None в Java).

Другой пример, это будет задача. Функция поиска в списке. Найти элемент в списке и вернуть его индекс. Если элемент есть, возвращается Just i, если элемента нет - возвращается Nothing.

## Тип Either
В типе Maybe есть значение Nothing, которое означает отстутствие значения. Но что если хочется не просто сказать, что значения нет, а еще сказать, почему его нет. Например, при чтении из файла функция чтения может вернуть строку с тем, что она прочитала, но может случиться ошибка, и тогда недостаточно просто вернуть Nothing, надо объяснить, что именно за ошибка случилась.

In [20]:
data Either' a b = Left' a | Right' b

Т.е. тип Either хранит либо значение одного типа, либо значение другого. `Either Int String` - это либо число (`Left Int`), либо строка (`Right String`).

Например, функция `stringToInt` может возвращать Either String Int, при этом если получилось строку превратить в число, возвращается число, если не получилось - возвращается исходная строка.

# Классы типов

In [21]:
add x y = x + y
:t add -- узнать тип add
:t (+) -- аналогично, узнать тип суммы

add :: forall a. Num a => a -> a -> a

(+) :: forall a. Num a => a -> a -> a

Мы получаем тип `Num a => a -> a -> a`, который ознчает, что эта функция из двух значения типа `a` получает еще одно значение этого типа. Но!!! Приписка `Num a => `, означает, что это работает не со всеми типами, а только с типами класса Num, т.е. с числовыми типами.

А как устроена фунция `min`?

In [22]:
:t min
:t minimum

-- сами сделаем функцию поиска минимума в списке
-- можно искать минимум, но только если элементы списка упорядочены
minimum' :: Ord a => [a] -> a
--minimum' [x] = x
--minimum' (x:xs) = min x $ minimum xs -- min x (minimum xs)
--minimum' list = foldl1 min list -- в fold1 можно не указывать начальный элемент
--minimum' = foldl1 min
minimum' = foldl1 (\x y -> if x < y then x else y)

minimum' [10, 20, 30, 5, 40, 50]

min :: forall a. Ord a => a -> a -> a

minimum :: forall (t :: * -> *) a. (Foldable t, Ord a) => t a -> a

5

Функция `<` умеет работать только с типами, которые имеют класс Ord. Технически, для каждого типа, для которого хочется, чтобы работала операция <, эта операция реализуется в явном виде. Т.е. в Haskell написал код, который выполняет < для чисел, отдельно написан код, который выполняет ее для строк и т.д. Мы тоже можем определить эту операцию для нашего собственного типа:

In [23]:
data Vector = Vector Int Int -- deriving Show

len2 :: Vector -> Int
len2 (Vector x y) = x * x + y * y

-- класс Eq это такие типы, элементы которых можно сравнивать ==
instance Eq Vector where
  v1 == v2 = len2 v1 == len2 v2

instance Ord Vector where
  --v1 < v2 = len2 v1 < len2 v2
  -- в классе Ord есть функции <, <=, >, >=, min, max, можно определять не все
  -- достаточно только определить <=. (но недостаточно только <, как мы пытались)
  v1 <= v2 = len2 v1 <= len2 v2

-- если не писать data ... deriving Show, можно самостоятельно опредрелить show
instance Show Vector where
  show (Vector x1 y1) = "(" ++ (show x1) ++ ", " ++ (show y1) ++ ")"

Vector 1 2 < Vector 2 3
Vector 1 2 >= Vector 2 3
minimum [Vector 1 2, Vector 3 4, Vector 0 1]

True

False

(0, 1)

Давайте определим какой-нибудь свой класс. Типы, элементы которых можно "переворачиавть"

In [24]:
class Reversable a where
  rev :: a -> a

Мы написали, что тип a является "Переворачиваемым", если для него есть метод `rev :: a -> a`. Теперь давайте научимся переворачивать числа и списки.

In [25]:
class Reversable a where
  rev :: a -> a
  
instance Reversable Int where
  -- rev x = list2num (reverse (num2list x)) where
  -- rev x = (list2num . reverse . num2list) x where
  rev = list2num . reverse . num2list where
          --list2num list = foldl (\a digit -> a*10 + digit) 0 list
          list2num = foldl (\a digit -> a*10 + digit) 0
          
          num2listHelper list 0 = list
          num2listHelper list x = num2listHelper (x1:list) x0 where
                                                               x0 = div x 10
                                                               x1 = mod x 10
          
          --num2list x = num2listHelper [] x
          num2list = num2listHelper []

-- приходится явно указывать тип для 42 (об этом далее)
rev (42::Int)

24

Давайте еще перевернем списки. Перевернутый список, это список, где элементы идут с конца в начало, при этом элементы внутри списка тоже перевернуты:

In [26]:
-- instance Reversable Int where
--instance Reversable [a] where
  --rev = foldl (\list e -> e:list) [] -- или можно по-другому:
  --rev = foldl (flip (:)) [] -- flip меняет местами аргументы функции, см. ранее

--опишем тип с типовым ограничителем
-- подходят не любые типы a, а только переворачиваемые
instance Reversable a => Reversable [a] where
  rev = foldl (\list e -> rev e:list) []

-- опять нужно подсказать Haskell, что список типа Int
rev [123, 42::Int]

[24,321]

### Проблемы класса Num

У числовых литералов нет конкретных типов Int, Double и т.п. Это просто какие-то типы класса Num или Fractional.
Элементы класса Num можно складывать, умножать, вычистать. Класс Fractional дополнительно позволяет делить. Класс Integral это целые числа, в них дополнительно появляется возможность считать остатки от деления. Эти классы определены так:
`class Num a => Fractional a`

In [27]:
:t 42
:t 2.39

42 :: forall p. Num p => p

2.39 :: forall p. Fractional p => p

Это сделано, чтобы числа разных типов можно было, например, складывать:
`1 + 1.5`. Если бы 1 имело тип `Int`, а 1.5 тип `Double`, то функция сложения не подошла бы по своим типам:

In [28]:
:t (+)

(+) :: forall a. Num a => a -> a -> a

Ей нужно два одинаковых типа, а `Int` и `Double` разные.

In [29]:
:t 1 + 1.5

1 + 1.5 :: forall a. Fractional a => a

Здесь 1 и 1.5 имеют тип `Fractional`, соответственно и результат тоже. Трудности возникают, когда тип `Int`, например, уже известен, и делить на него нельзя:

In [30]:
list = [10, 20, 30, 40]
length list
:t length list -- узнаем тип числа
mean = sum list / length list

4

length list :: Int

<interactive>:1:8: error: • No instance for (Fractional Int) arising from a use of ‘/’ • In the expression: sum list / length list In an equation for ‘mean’: mean = sum list / length list

Но поделить хочется. Приходится пользоваться функцией `fromIntegral`:

In [31]:
:t fromIntegral

fromIntegral :: forall a b. (Integral a, Num b) => a -> b

Как понять тип? Она превращает целые числа (Integral - это "подкласс" Num), в Num

In [32]:
list = [10, 20, 30, 40]
mean = (fromIntegral $ sum list) / (fromIntegral $ length list)
mean

25.0